In [1]:
import numpy as np
aX = np.array([[[1, 2, 3, 4], [4,5,6, 7]], [[7,8,9, 10],[10,11,12, 13]]])
print(aX)

aY = aX[:, :, [2,1,0,3]]
print(aY)


[[[ 1  2  3  4]
  [ 4  5  6  7]]

 [[ 7  8  9 10]
  [10 11 12 13]]]
[[[ 3  2  1  4]
  [ 6  5  4  7]]

 [[ 9  8  7 10]
  [12 11 10 13]]]


In [ ]:
import numpy as np


aX = np.array([[[1, 2, 3], [4,5,6]], [[7,8,9],[10,11,12]]])
print(aX)
aXlin = aX.reshape(4, 3)

aY = np.array([[1,1,1], [2,2,2]])

aXex = np.expand_dims(aXlin, axis=0)
aYex = np.expand_dims(aY, axis=1)
aZex = aXex - aYex

In [ ]:
aA = np.array([1,2,3])
aB = np.array([aA])
fX = np.dot(aA, aA)

for aX in aB:
    print(aX)
# endfor

In [3]:
import numpy as np
from anycam.model.cls_camera_lut import CCameraLut

xCamLut = CCameraLut()
xCamLut.FromFile(_xFilePath="[path]/image-render-workspace-test/_render/RayMap/Pinhole/Image0001.exr",
                    _iLutBorderPixel=0, _iLutSuperSampling=1)

print(xCamLut.imgLut.shape)

(100, 100, 4)


In [4]:
aRayDirs = xCamLut.imgLut[:, :, 0:3]
iRows, iCols, _ = aRayDirs.shape
aGridR, aGridC = np.mgrid[0:iRows, 0:iCols]

aGrid = np.concatenate((aGridR[:, :, np.newaxis], aGridC[:, :, np.newaxis]), axis=2)

iPixCnt = iRows * iCols
aDirs = aRayDirs.reshape(iPixCnt, 3)
aPos = aGrid.reshape(iPixCnt, 2)



In [21]:
import math
fA = math.radians(15.0)
# aTestDir = np.array([[math.sin(fA), 0.1, -math.cos(fA)]])
aA = aRayDirs[5, 10, :]
aB = aRayDirs[5, 11, :]
aTestDir = np.expand_dims(aA + 0.5 * (aB - aA), axis=0)

# aTestDir = aRayDirs[5, 10:12, :]
aTestDirEx = np.expand_dims(aTestDir, axis=1)
aDirsEx = np.expand_dims(aDirs, axis=0)

aSub = aDirsEx - aTestDirEx
aDist = np.linalg.norm(aSub, axis=2)
aMinIdx = np.argmin(aDist, axis=1)
# aDistSort = np.argsort(aDist, axis=1)

# aSelIdx = aDistSort[:, 0:4]



In [22]:
# for iTestIdx, iMinIdx in enumerate(aMinIdx):
iTestIdx = 0
iMinIdx = aMinIdx[iTestIdx]

iRowIdx = iMinIdx // iRows
iColIdx = iMinIdx % iRows
print(f"{iRowIdx}, {iColIdx}")

iColMin = max(0, iColIdx - 1)
iColMax = min(iCols-1, iColIdx + 1)
iRowMin = max(0, iRowIdx - 1)
iRowMax = min(iRows-1, iRowIdx + 1)

aTest = aTestDir[iTestIdx]
aNearest = aRayDirs[iRowIdx, iColIdx, :]
aOrig = aRayDirs[iRowMin, iColMin, :]
aMaxX = aRayDirs[iRowMin, iColMax, :]
aMaxY = aRayDirs[iRowMax, iColMin, :]

aX = (aMaxX - aOrig) / (iColMax - iColMin)
aY = (aMaxY - aOrig) / (iRowMax - iRowMin)

fXlen = np.linalg.norm(aX)
fYlen = np.linalg.norm(aY)

aDelta = aTest - aNearest
fDeltaPixX = np.dot(aDelta, aX) / (fXlen*fXlen)
fDeltaPixY = np.dot(aDelta, aY) / (fYlen*fYlen)

fPixRow = iRowIdx + fDeltaPixY
fPixCol = iColIdx + fDeltaPixX

print(f"{fPixRow}, {fPixCol}")

# endfor

5, 10
4.9535911081767265, 10.501891336234213


In [ ]:
import math
from scipy.interpolate import griddata

# aTestDir = aRayDirs[5, 10:11, :]
fA = math.radians(15.0)
aTestDir = np.array([[math.sin(fA), 0.1, -math.cos(fA)]])

aPixPos = griddata(aDirs, aPos, aTestDir, method="linear")
